In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_v2_behavior()
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
from networks import C_VAE_NET,D_VAE_NET

from utils import ones_target, zeros_target
import numpy as np
from dual_1_contrastive import full_model
import timeit
start = timeit.default_timer()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
cd ..

C:\Users\gghos\Downloads\IGNITE


Load datasets

In [3]:
data_path = 'final_physio/datasets/'
with open(os.path.join(data_path, 'FP_2012_mask.pkl'), 'rb') as f:
    miss = pickle.load(f)
# with open(os.path.join(data_path, 'P2012_age_conditions.pkl'), 'rb') as f:
#     statics_label = pickle.load(f)
with open(os.path.join(data_path, 'FP_2012_LOCF.pkl'), 'rb') as f:
    LVCF = pickle.load(f)
with open(os.path.join(data_path, 'FP_2012_zero.pkl'), 'rb') as f:
    zero = pickle.load(f)
with open(os.path.join(data_path, 'FP_2012_Intervention.pkl'), 'rb') as f:
    interventions = pickle.load(f)

In [4]:
with open(os.path.join('final_physio/person.pkl'), 'rb') as f:
    mask_personalized = pickle.load(f)

pass data

In [5]:
sample_size= len(LVCF)
# params:

# statics_label = statics_label2[:sample_size]
statics_label = np.asarray(interventions)[:sample_size]
# [:, :].reshape([-1, 1])
# discrete_x = np.concatenate([mean[:sample_size],mask_personalized[:sample_size]], axis = 2)
discrete_x = np.multiply(LVCF[:sample_size],mask_personalized[:sample_size])
# discrete_x = LVCF[:sample_size]
continuous_x = zero[:sample_size]
miss_sample_x = miss[:sample_size]
# PM_sample_x = mask_personalized[:sample_size]

In [6]:
batch_size = 400
time_steps = continuous_x.shape[1]
num_pre_epochs = 200

shared_latent_dim = 10
c_z_size = shared_latent_dim
c_dim = continuous_x.shape[2]

d_dim = discrete_x.shape[2]
d_z_size = shared_latent_dim

conditional = True
num_labels = 1

In [7]:
statics_label=np.where(np.isnan(statics_label), 0, statics_label)

In [8]:

# networks for continuousVAE
c_vae = C_VAE_NET(batch_size=batch_size, time_steps=time_steps, dim=c_dim, z_dim=c_z_size,
                  conditional=conditional, num_labels=num_labels)

d_vae = D_VAE_NET(batch_size=batch_size, time_steps=time_steps, dim=d_dim, z_dim=d_z_size,
                  conditional=conditional, num_labels=num_labels)


# create data directory for saving
checkpoint_dir = os.path.join("MyDrive/physionet.org/", "data/checkpoint/")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [9]:
exp_name = "IGNITE_dual_single_contrastive_500_treat_dot_IMM_10_001"

In [10]:
tf.compat.v1.reset_default_graph()
run_config = tf.compat.v1.ConfigProto()
with tf.compat.v1.Session(config = run_config) as sess:
        model = full_model(sess=sess,
                      batch_size=batch_size,
                      time_steps=time_steps,
                      num_pre_epochs=num_pre_epochs,
                      checkpoint_dir=checkpoint_dir,
                      c_dim=c_dim, 
                      c_z_size=c_z_size, c_data_sample=continuous_x,
                      c_vae=c_vae, 
                      d_dim=d_dim, 
                      d_z_size=d_z_size, d_data_sample=discrete_x,
                      d_vae=d_vae,
                      # PM_data_sample = PM_sample_x,
                      mask_data_sample=miss_sample_x,experiment_name=exp_name,
                      conditional=conditional, 
                       num_labels=num_labels,
                      statics_label=statics_label)
        model.build()
        model.train()

stop = timeit.default_timer()
print('Time: ', stop - start)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(400, 48, 36)
finalize
data_size 4000
start training
VAE epoch 0
10
VAE epoch 1
10
VAE epoch 2
10
VAE epoch 3
10
VAE epoch 4
10
VAE epoch 5
10
VAE epoch 6
10
VAE epoch 7
10
VAE epoch 8
10
VAE epoch 9
10
VAE epoch 10
10
VAE epoch 11
10
VAE epoch 12
10
VAE epoch 13
10
VAE epoch 14
10
VAE epoch 15
10
VAE epoch 16
10
VAE epoch 17
10
VAE epoch 18
10
VAE epoch 19
10
VAE epoch 20
10
VAE epoch 21
10
VAE epoch 22
10
VAE epoch 23
10
VAE epoch 24
10
VAE epoch 25
10
VAE epoch 26
10
VAE epoch 27
10
VAE epoch 28
10
VAE epoch 29
10
VAE epoch 30
10
VAE epoch 31
10
VAE epoch 32
10
VAE epoch 33
10
VAE epoch 34
10
VAE epoch 35
10
VAE epoch 36
10
VAE epoch 37
10
VAE epoch 38
10
VAE epoch 39
10
VAE epoch 40
10
VAE epoch 41
10
VAE epoch 42
10
VAE epoch 43
10
VAE epoch 44
10
VAE epoch 45
10
VAE epoch 46
10
VAE epoch 47
10
VAE epoch 48
10
VAE epoch 49
10
VAE epoch 50
10
VAE epoch 51
10
VAE epo

In [11]:
%load_ext tensorboard
%tensorboard --logdir logs/last

Reusing TensorBoard on port 6006 (pid 4632), started 3 days, 2:49:50 ago. (Use '!kill 4632' to kill it.)

In [12]:
round(4000/256)

16